In [ ]:
import sys
print(sys.executable)

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

print(torch.__version__)


In [ ]:
x = torch.tensor([2.0, 3.], requires_grad=True).to(torch.float32)
y = x.sum() * 2
print(f"y: {y}")
y.backward()
print("y: ", y)
print("y grad:", y.retain_grad())
print("x grad:", x.grad)

In [ ]:
weight = 0.5
bias = 1.5

start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step) #unsqueeze(dim=1 )
y = weight * X + bias
print(X[:10], y[:10])

In [ ]:
X.__len__(), y.__len__()

In [ ]:
train_split = 0.8
test_split = 0.2

X_train_split_int, X_test_split_int = int(train_split * len(X)), int(test_split * len(X))
y_train_split_int, y_test_split_int = int(train_split * len(y)), int(test_split * len(y))

In [ ]:
X_train = X[:X_train_split_int]
X_test = X[:X_test_split_int]
y_train = y[:y_train_split_int]
y_test = y[:y_test_split_int]

Visualize

In [ ]:
def plot(train_data=X_train, train_labels=y_train, test_data=X_test, test_labels=y_test, predictions=None):
    plt.figure(figsize=(10, 7))
    plt.scatter(train_data, train_labels, c="b", s=4, label="training")
    plt.scatter(test_data, test_labels, c="g", s=4, label="testing")
    
    if predictions is not None:
        plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")
        
    plt.legend(prop={"size": 14})

In [ ]:
plot()

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weights = nn.Parameter(torch.rand(1, requires_grad=True, dtype=torch.float))
        self.bias = nn.Parameter(torch.rand(1, requires_grad=True, dtype=torch.float))
        
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias
    pass

In [ ]:
torch.rand(1).dim()

In [ ]:
l = LinearRegressionModel()
list(l.parameters())

In [ ]:
l.state_dict()

In [ ]:
with torch.inference_mode():
    y_preds = l(X_test)
    
y_preds
#plot(predictions=y_preds)
print(y_preds.data)
print(l.weights)
print(l.bias)

In [ ]:
l.state_dict()

In [ ]:
with torch.inference_mode():
    y_preds = l(X_test)
plot(predictions=y_preds)
print(y_preds.data, X_test.data)

In [ ]:
loss_fn = nn.L1Loss(reduction="mean")
optimizer = torch.optim.SGD(params=l.parameters(), lr=0.01)

In [ ]:
next(l.parameters()).data == l.state_dict()["weights"].data

In [ ]:
epoch_count = []
loss_values = []
test_loss_values = []

epochs = 100
for epoch in range(epochs):
    l.train()
    
    #forward pass
    y_preds = l(X_train)
    
    #calculate loss
    loss = loss_fn(y_preds, y_train)
    #print(f"Loss: {loss}")
    #optimizer zero grad
    optimizer.zero_grad()
    
    
    #backward pass
    loss.backward()
    #print(f"Weight gradient: {l.weights.grad}, Bias gradient: {l.bias.grad}")
    #step the optimizer
    
    optimizer.step()
    
    l.eval() # turn off unrequired settings (dropout, batchnorm, etc.)
    
    with torch.inference_mode():
        
        if epoch % 10 == 0:
            epoch_count.append(epoch)
            loss_values.append(loss.item())
            y_test_preds = l(X_test)
            test_loss = loss_fn(y_test_preds, y_test)
            test_loss_values.append(test_loss.item())
            print(f"Test loss: {test_loss}, Epoch: {epoch+1}/{epochs}")
    
plot(predictions=y_test_preds)

In [ ]:
plt.plot(epoch_count, loss_values, label="Train Loss")
plt.plot(epoch_count, test_loss_values, label="Test Loss")
plt.title("Loss vs Epochs")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

In [ ]:
weight, bias

In [ ]:
l.state_dict()["weights"], l.state_dict()["bias"]

In [ ]:
#torch.save(), torch.load(), torch.nn.Module.load_state_dict()


In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "01_pytorch_workflow_model.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(obj=l.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
!ls -l models

In [ ]:
load_model_l = LinearRegressionModel()
load_model_l.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [ ]:
load_model_l.state_dict()

In [ ]:
load_model_l.state_dict()["weights"] == l.state_dict()["weights"]

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
weight = 0.7
bias = 0.3
start = 0
end = 10
step = 0.1

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias
print(X[:10], y[:10])


In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
plot(X_train, y_train, X_test, y_test)

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_Layer = nn.Linear(in_features=1, out_features=1)
        
    def forward(self, x: torch.Tensor):
        return self.linear_Layer(x)

In [ ]:
torch.manual_seed(42)
model_1 = LinearRegressionModel()
model_1, model_1.state_dict()

In [ ]:
next(model_1.parameters()).device

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_1.parameters(), lr=0.01)

In [ ]:
epoch_count = []
loss_values = []
epochs = 100

for epoch in range(epochs):
    model_1.train()
    
    y_preds = model_1(X_train)
    
    loss = loss_fn(y_preds, y_train)
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model_1.eval()
    
    with torch.inference_mode():
        if epoch % 10 == 0:
            test_pred = model_1(X_test)
            
            test_loss = loss_fn(test_pred, y_test)
            
            print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")

In [ ]:
from pprint import pprint
pprint(model_1.state_dict())
print(f"weight:{weight}, bias:{bias}")

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "01_pytorch_workflow_model_self.pth"

MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
print(MODEL_SAVE_PATH)
torch.save(obj=model_1.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
loaded_model_1 = LinearRegressionModel()
loaded_model_1.load_state_dict(torch.load(MODEL_SAVE_PATH))

print(loaded_model_1)

In [ ]:
loaded_model_1.eval()

with torch.inference_mode():
    test_pred = model_1(X_test)
    y_preds_loaded = loaded_model_1(X_test)
test_pred == y_preds_loaded

# hitting extra lecture

In [ ]:
import torch
from torch import nn

In [ ]:
weight = 0.5
bias = 0.2
start = 1
end = 10
step = 0.1

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

print(f"X: {X[:10]}, y: {y[:10]}")

In [ ]:
X_train_split_int, X_test_split_int = int(0.8 * len(X)), int(0.2 * len(X))
y_train_split_int, y_test_split_int = int(0.8 * len(y)), int(0.2 * len(y))

X_train = X[:X_train_split_int]
X_test = X[X_train_split_int:]
y_train = y[:y_train_split_int]
y_test = y[y_train_split_int:]

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [ ]:
model_self = LinearRegressionModel()
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_self.parameters(), lr=0.01)

In [ ]:
epochs = 1000

for epoch in range(epochs):
    model_self.train()
    
    y_preds = model_self(X_train)
    
    loss = loss_fn(y_preds, y_train)
    
    optimizer.zero_grad()
    
    loss.backward()
    
    optimizer.step()
    
    model_self.eval()
    
    with torch.inference_mode():
        if epoch % 10 == 0:
            test_pred = model_self(X_test)
            test_loss = loss_fn(test_pred, y_test)
            print(f"Epoch: {epoch} | Train loss: {loss} | Test loss: {test_loss}")
    

# other stuff

**loss**, *loss*

In [ ]:
def wrapper(func):
    def logger(*args):
        print(f"logged {func.__name__}")
        return func()
    return logger

def function_1():
    print("function_1 was called")

In [ ]:
wrapper(function_1)

In [ ]:
function_1 = wrapper(function_1)

In [ ]:
function_1()

In [ ]:
text = (b"hello")

In [ ]:
type(text)

In [ ]:
import threading
import time

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class BasicNet(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.net = nn.Linear(5, 3, 1)
        
    def forward(self, x):
        x = nn.functional.relu(self.net(x))
        print(self.net(x))
        return self.net(x)

In [ ]:
BasicNet().state_dict()

In [ ]:
import dis

def add(a, b):
    return a + b

dis.dis(add)


In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class Stack(Generic[T]):
    def __init__(self):
        self.items: list[T] = []

In [ ]:
type(T)

In [ ]:
t = torch.tensor((2,2))
t

In [ ]:
t_rand = torch.rand((2,2, 2), requires_grad=True)
t_rand, t_rand.dim()

In [ ]:
t + t_rand

In [ ]:
t.shape, t_rand.shape

In [ ]:
t_rand.min()

In [ ]:
print(t.grad)

In [ ]:

type(t_rand.grad)

In [ ]:
y = t**2 - t_rand **3

In [ ]:
y

In [ ]:
y.backward()

In [ ]:
a = torch.tensor([5.], requires_grad=True)
b = torch.tensor([3.], requires_grad=True)

y = a**2 - b**2
y.backward()
print(a.grad, b.grad)

In [ ]:
a.grad.data == a.grad

In [ ]:
x = torch.rand(1, 10,  requires_grad=True)
w = torch.rand(10, 1,  requires_grad=True)
b = torch.rand(1,  requires_grad=True)

In [ ]:
w, x

In [ ]:
y = torch.matmul(x, w) + b

In [ ]:
y.shape

In [ ]:
loss = 1 - y
loss

In [ ]:
loss.backward()
w.grad

In [ ]:
with torch.no_grad():
    w = w - 0.001 * w.grad.data

In [ ]:
#
